In [122]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt 
#import seaborn as sns
from sklearn.naive_bayes import GaussianNB
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [123]:
train = pd.read_csv("pre_processed_train.csv")
test = pd.read_csv("pre_processed_test.csv")

In [124]:
t1 = train

In [125]:
ytrain = train["isFraud"]
train.drop(axis = 1, labels = ["isFraud", "Unnamed: 0"], inplace = True)

In [126]:
test.drop(axis = 1, labels = ["Unnamed: 0"], inplace = True)

In [127]:
over = SMOTE(sampling_strategy = 0.15)
under = RandomUnderSampler(sampling_strategy = 0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
train1, ytrain1 = pipeline.fit_resample(train, ytrain)

In [128]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train = scale.fit_transform(train1)
X_test = scale.transform(test) 

In [129]:
grid_params = {  
#'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
'var_smoothing': np.logspace(0, -9, num = 100)
} 
gs = GridSearchCV(GaussianNB(), grid_params, verbose = 1, cv = 3, n_jobs = -1, scoring = 'roc_auc')
gs = gs.fit(X_train,ytrain1)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [130]:
print("Tuned Hyperparameters :", gs.best_params_)
print("Accuracy :", gs.best_score_)

Tuned Hyperparameters : {'var_smoothing': 1.0}
Accuracy : 0.8135812723119576


In [131]:
y_pred1 = gs.predict(X_test)
y_pred1.sum()

27131

In [132]:
import csv
with open('naive.csv', 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "isFraud"])
    idx = 0
    for i in y_pred1:
        writer.writerow([idx, i])
        idx = idx + 1